# Exercise 2#

## Task 1 ##


a) Tokenize the sentences in fileyelp_polarity.txt, removing punctuation.

In [33]:
import string
import collection
import functools
import operator 
import pandas as pd
import numpy as np
import os

def read_file(filename):
    df = pd.read_csv(filename, sep="\t", header=None)
    df = df.assign(tokens =df[0].apply(lambda x : x.translate(str.maketrans('', '', string.punctuation)).lower().split()))
    return df

df = read_file("yelp_polarity.txt")
display(df)

ImportError: No module named 'collection'

The file is read using the function read_file. Read_file uses pandas to read the file and saves it in a dataframe. A function is then applied to all of the sentences using the apply() funciton and saved in a new column titled "tokens". The lambda-function used first removes punctuation, then applies lower() and split(). This ensures that puntuaction is not included in the tokens and removal of capitalizatrion does not lead to two tokens being considered different when written using a different case.

b) Calculate token and bigram counts

In [11]:
def ngram_to_ngram_freq(tokens):
    dictionary = dict.fromkeys(tokens)
    for i in list(dictionary):
        count = 0
        for j in tokens:
            if j==i:
                count += 1
        dictionary[i] = count
    return dictionary

def bigram_list(sent):
    bigrams=[]
    for i in range(0, len(sent) - 1):
        bigrams.append(sent[i] + ' ' + sent[i + 1])
    return bigrams

df = df.assign(bigrams = df["tokens"].apply(lambda x : dict.fromkeys(bigram_list(x))))
df = df.assign(token_count = df["tokens"].apply(lambda x : ngram_to_ngram_freq(x)))
df = df.assign(bigram_count = df["bigrams"].apply(lambda x : ngram_to_ngram_freq(x)))
display(df)

,0,1,tokens,bigrams,token_count,bigram_count
0,Wow... Loved this place.,1,"[wow, loved, this, place]","{'loved this': None, 'this place': None, 'wow ...","{'loved': 1, 'place': 1, 'wow': 1, 'this': 1}","{'loved this': 1, 'this place': 1, 'wow loved'..."
1,Crust is not good.,0,"[crust, is, not, good]","{'crust is': None, 'is not': None, 'not good':...","{'not': 1, 'good': 1, 'crust': 1, 'is': 1}","{'crust is': 1, 'is not': 1, 'not good': 1}"
2,Not tasty and the texture was just nasty.,0,"[not, tasty, and, the, texture, was, just, nasty]","{'was just': None, 'and the': None, 'tasty and...","{'the': 1, 'tasty': 1, 'just': 1, 'nasty': 1, ...","{'was just': 1, 'and the': 1, 'tasty and': 1, ..."
3,Stopped by during the late May bank holiday of...,1,"[stopped, by, during, the, late, may, bank, ho...","{'may bank': None, 'rick steve': None, 'late m...","{'stopped': 1, 'during': 1, 'the': 1, 'may': 1...","{'may bank': 1, 'rick steve': 1, 'late may': 1..."
4,The selection on the menu was great and so wer...,1,"[the, selection, on, the, menu, was, great, an...","{'the prices': None, 'so were': None, 'was gre...","{'so': 1, 'prices': 1, 'menu': 1, 'selection':...","{'the prices': 1, 'was great': 1, 'great and':..."
...,...,...,...,...,...,...
995,I think food should have flavor and texture an...,0,"[i, think, food, should, have, flavor, and, te...","{'food should': None, 'i think': None, 'and te...","{'texture': 1, 'think': 1, 'i': 1, 'and': 2, '...","{'food should': 1, 'i think': 1, 'and texture'..."
996,Appetite instantly gone.,0,"[appetite, instantly, gone]","{'appetite instantly': None, 'instantly gone':...","{'appetite': 1, 'instantly': 1, 'gone': 1}","{'appetite instantly': 1, 'instantly gone': 1}"
997,Overall I was not impressed and would not go b...,0,"[overall, i, was, not, impressed, and, would, ...","{'was not': None, 'would not': None, 'overall ...","{'would': 1, 'i': 1, 'back': 1, 'go': 1, 'over...","{'was not': 1, 'not go': 1, 'would not': 1, 'i..."
998,"The whole experience was underwhelming, and I ...",0,"[the, whole, experience, was, underwhelming, a...","{'just go': None, 'i think': None, 'ninja sush...","{'go': 1, 'next': 1, 'the': 1, 'sushi': 1, 'ex...","{'just go': 1, 'i think': 1, 'whole experience..."


The function ngram_to_ngram_freq takes a list of ngrams as a paramater and returns a dictionary with distinct ngrams as keys and the individual frequencies in the list of ngrams. 

The function bigram_list determines all the bigrams that occur in a given tokenized sentence (that retains the word order of the untokenized sentence!) passed as a parameter. This is done by iterating through the tokens and saving all bigrams.

To complete the task, the column "bigrams" of all bigrams that exist in each sentence is added to the dataframe from the first task by applying the bigram_list function to the column "tokens". Then two additional coloums are created, "token_count" and "bigram_count" which contain the result of ngram_to_ngram_freq after being passed the "tokens" and the "bigrams" column respectively.

c) Print the 10 most frequent bigrams and their frequency

In [12]:
def frequency(column):
    frequency = dict(functools.reduce(operator.add, map(collections.Counter, column)))
    frequency = sorted(frequency.items(), key=lambda x: x[1])
    return(list(frequency))

bigrams = frequency(df["bigram_count"])
print(bigrams[len(bigrams) - 10 : ])

[('on the', 23), ('the service', 24), ('service was', 26), ('and i', 27), ('of the', 29), ('food was', 32), ('and the', 38), ('it was', 41), ('the food', 47), ('this place', 72)]


The function freuquency takes the dataframe column passed as a parameter, which has to consist of dictionary of words and their freuquecies per row, and calculates the overall frequency in the whole dataframe sorted in asecneding order by the frequency. The result is a list of bigrams and their frequency.

This function is called on he colum "bigram_count" created in the previous task. To find the ten most frequent bigrms the last ten elements of the list are printed.

d.) Print a bigram matrix from the 10 most frequent tokens

In [32]:
token_frequency = frequency(df["token_count"])
def bigram_matrix(token_list, bigram_frequencies):
    matrix = np.zeros((len(token_list), len(token_list)))
    bigram_frequencies = dict(bigram_frequencies)
    for i in range(0, len(matrix)):
        for j in range(0, len(matrix)):
            key_str = token_list[i] + " " + token_list[j]
            if key_str in bigram_frequencies:
                matrix[i, j] = bigram_frequencies[key_str] 
    return matrix
freq_matrix = bigram_matrix(list(dict(token_frequency[len(token_frequency) - 10 : ]).keys()), bigrams)
token_strings = dict(token_frequency[len(token_frequency) - 10 : ]).keys()
display(pd.DataFrame(freq_matrix, token_strings, token_strings))

,it,and,the,is,a,this,was,of,i,to
it,0.0,3.0,0.0,10.0,1.0,0.0,41.0,0.0,2.0,2.0
and,17.0,0.0,38.0,0.0,3.0,4.0,5.0,0.0,27.0,0.0
the,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
is,0.0,0.0,10.0,0.0,16.0,0.0,1.0,1.0,0.0,1.0
a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
this,0.0,0.0,0.0,20.0,0.0,0.0,10.0,0.0,0.0,1.0
was,0.0,0.0,13.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0
of,1.0,0.0,29.0,0.0,10.0,4.0,0.0,0.0,0.0,0.0
i,0.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0
to,1.0,1.0,11.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0


First the most frequent tokens are determined using the previously implemented frequency fucntion.

Then the function bigram_matrix is defined. This d=function takes a list of tokens and a list of all possible bigrams wit their respective frequencies as input. A two dimensional array (matrix) is createn with the demensions given by the number of tokens in the token_list. With a doulbe for-loop the function then iterates over all possible bigrams that can be created with the the token_list and saves the frequency of that bigram in the matrix.

This function is called and given the ten most frequent tokens as well as the previously created dictionary bigrams. The return matrix is then read into an new pandas Data frame and displayed.

## Task 2

Using the code of Task 1:

a.) Extract top 20 most frequent bigrams of all positive and negative documents in folder polarity




In [29]:
pos_docs = os.listdir("./polarity/pos")
pos_df = read_file("./polarity/pos/" + pos_docs[0])
for doc in pos_docs[1:]:
    pos_df = pos_df.append(read_file("./polarity/pos/" + doc))

pos_df = pos_df.assign(bigrams = pos_df["tokens"].apply(lambda x : dict.fromkeys(bigram_list(x))))
pos_df = pos_df.assign(token_count = pos_df["tokens"].apply(lambda x : tokens_to_token_freq(x)))
pos_df = pos_df.assign(bigram_count = pos_df["bigrams"].apply(lambda x : tokens_to_token_freq(x)))

pos_bigrams = frequency(pos_df["bigram_count"])
print(pos_bigrams[len(pos_bigrams) - 20:])

neg_docs = os.listdir("./polarity/neg")
neg_df = read_file("./polarity/neg/" + neg_docs[0])
for doc in neg_docs[1:]:
    neg_df = neg_df.append(read_file("./polarity/neg/" + doc))

print()
neg_df = neg_df.assign(neg_bigrams = neg_df["tokens"].apply(lambda x : dict.fromkeys(bigram_list(x))))
neg_df = neg_df.assign(token_count = neg_df["tokens"].apply(lambda x : tokens_to_token_freq(x)))
neg_df = neg_df.assign(bigram_count = neg_df["neg_bigrams"].apply(lambda x : tokens_to_token_freq(x)))
# print(neg_df)
neg_bigrams = frequency(neg_df["bigram_count"])
print(neg_bigrams[len(neg_bigrams) - 20:])
# print(neg_df[0])

[('for the', 4), ('there is', 4), ('the story', 4), ('it is', 5), ('the first', 5), ('as the', 5), ('the movie', 5), ('this film', 5), ('from the', 5), ('and the', 5), ('is a', 6), ('is the', 7), ('with a', 7), ('to be', 7), ('by the', 8), ('with the', 8), ('the film', 8), ('the shark', 9), ('of the', 14), ('in the', 24)]

[('they are', 4), ('the story', 4), ('is not', 5), ('from the', 5), ('of a', 5), ('and the', 5), ('by the', 5), ('it is', 6), ('in a', 6), ('with the', 6), ('the movie', 6), ('for the', 6), ('film is', 6), ('of course', 6), ('on the', 7), ('in the', 8), ('to be', 8), ('that the', 8), ('the film', 9), ('of the', 10)]


b.) Calculate their probability as aforementioned

In [30]:
top_pos_bigrams = pos_bigrams[len(pos_bigrams) - 20:]
top_neg_bigrams = neg_bigrams[len(neg_bigrams) - 20:]

def calculate_probabilty(all_bigrams, bigrams):
    probability = []
    for x in bigrams:
        probability.append((x[0], x[1]/sum(dict(all_bigrams).values())))
    return probability

neg_probability = calculate_probabilty(neg_bigrams, top_neg_bigrams)
print("Negative Bigram Probabilities")
print(neg_probability)
print()

pos_probability = calculate_probabilty(pos_bigrams, top_pos_bigrams)
print("Positive Bigram Probabilities")
print(pos_probability)

Negative Bigram Probabilities
[('they are', 0.0016326530612244899), ('the story', 0.0016326530612244899), ('is not', 0.0020408163265306124), ('from the', 0.0020408163265306124), ('of a', 0.0020408163265306124), ('and the', 0.0020408163265306124), ('by the', 0.0020408163265306124), ('it is', 0.0024489795918367346), ('in a', 0.0024489795918367346), ('with the', 0.0024489795918367346), ('the movie', 0.0024489795918367346), ('for the', 0.0024489795918367346), ('film is', 0.0024489795918367346), ('of course', 0.0024489795918367346), ('on the', 0.002857142857142857), ('in the', 0.0032653061224489797), ('to be', 0.0032653061224489797), ('that the', 0.0032653061224489797), ('the film', 0.003673469387755102), ('of the', 0.004081632653061225)]

Positive Bigram Probabilities
[('for the', 0.001234186979327368), ('there is', 0.001234186979327368), ('the story', 0.001234186979327368), ('it is', 0.0015427337241592102), ('the first', 0.0015427337241592102), ('as the', 0.0015427337241592102), ('the mov

c.) Write a script to save in a file the most probable bigrams of both classes and their probabilites separated by tabs

In [27]:
neg_prob = dict(neg_probability)
pos_prob = dict(pos_probability)

df_neg = pd.DataFrame(neg_prob.values(),neg_prob.keys())
df_neg.columns = ['neg prob']
df_pos =  pd.DataFrame(pos_prob.values(), pos_prob.keys())
df_pos.columns = ['pos prob']
df_total = pd.concat([df_neg, df_pos], axis=1, sort=True)

display(df_total)
df.to_csv("test1.txt", sep="\t")

,neg prob,pos prob
and the,0.002041,0.001543
as the,NaN,0.001543
by the,0.002041,0.002468
film is,0.002449,NaN
for the,0.002449,0.001234
from the,0.002041,0.001543
in a,0.002449,NaN
in the,0.003265,0.007405
is a,NaN,0.001851
is not,0.002041,NaN


d.) Compare the output of both classes and write an analysis of your obersvations

It is clear that the 10 most frequent bigrams in pos aren't the same top 10 in neg, even though several are in the top 10 of both. It is also clear that many of the bigrams that made into the top 10 of both datasets consit of the most common English words such as "with the" or "and the". More rare bigrams such as "the shark" or "film is" only appear in the top 10 of one of the datasets. However, the frequencies of the top 10 bigrams don't reveal a lot, since the vast majority consist of very common English words/bigrams that can be used in almost every context. The fact that the word "film" appears in multiple bigrams in both datasets and "movie" also apears in both datasets, suggest that the dataset has something to do with ratings of a movie/film. Considering this, it is also worht noting that bigrams with very similiar meaning, such as "the film" and "the movie" are not combined and don't have a combined frequency even though they are synonymous. This show us that further methods are needed to really gain meaningful data from this (or other) datasets. It is also worth noting that even the highest frequencies are fairy low, but this isn't surprising when considering all the possible bigrams contained in a data set.